In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

X_train = pd.read_csv('./train.csv')
X_test = pd.read_csv('./test.csv')

print(X_train)
print(X_test)


          ID  Gender Ever_Married  Age Graduated  Profession  Work_Experience  \
0     462809    Male           No   22        No  Healthcare              1.0   
1     466315  Female          Yes   67       Yes    Engineer              1.0   
2     461735    Male          Yes   67       Yes      Lawyer              0.0   
3     461319    Male          Yes   56        No      Artist              0.0   
4     460156    Male           No   32       Yes  Healthcare              1.0   
...      ...     ...          ...  ...       ...         ...              ...   
6660  463002    Male          Yes   41       Yes      Artist              0.0   
6661  464685    Male           No   35        No   Executive              3.0   
6662  465406  Female           No   33       Yes  Healthcare              1.0   
6663  467299  Female           No   27       Yes  Healthcare              1.0   
6664  461879    Male          Yes   37       Yes   Executive              0.0   

     Spending_Score  Family

In [3]:
# EDA
# shape
print(X_train.shape)
print(X_test.shape)


# info
print(X_train.info())
print(X_test.info())

# describe

print(X_train.describe())
print(X_test.describe())


# ID는 삭제시킨다. 그리고 범주형 변수는 라벨인코딩이나, 원핫인코딩을 돌릴 것이다.
# target변수는 Segmentation은 다른 파일을 제작한다.

# isnull : 결측치는 없음

# 범주형 변수 빈도 확인

for col in X_train.select_dtypes(object).columns.tolist():
    print('-'*80)
    print(col)
    print(X_train[col].unique())
    print(X_train[col].value_counts())
for col in X_test.select_dtypes(object).columns.tolist():
    print('-'*80)
    print(col)
    print(X_test[col].unique())
    print(X_test[col].value_counts())


(6665, 11)
(2154, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   

In [4]:
# 전처리
# 더미 : var_1, Profession
# 라벨링 : Gender, Ever_Married, Graduated, Spending Score

# 필요없는 컬럼 정리
X_test_ID = X_test['ID']
y_train = X_train['Segmentation']
X_train.drop(columns=['ID', 'Segmentation'], inplace=True)
X_test.drop(columns='ID', inplace=True)

# 범주형 변수 정리
# 라벨링
'''
X_train['Gender'] = X_train['Gender'].map({'Male':1, 'Female':0})
X_test['Gender'] = X_test['Gender'].map({'Male':1, 'Female':0})
'''

X_train['Ever_Married'] = X_train['Ever_Married'].map({'No':0, 'Yes':1})
X_test['Ever_Married'] = X_test['Ever_Married'].map({'No':0, 'Yes':1})

X_train['Graduated'] = X_train['Graduated'].map({'No':0, 'Yes':1})
X_test['Graduated'] = X_test['Graduated'].map({'No':0, 'Yes':1})


X_train['Spending_Score'] = X_train['Spending_Score'].map({'Low':0, 'Average':1, 'High':2})
X_test['Spending_Score'] = X_test['Spending_Score'].map({'Low':0, 'Average':1, 'High':2})


# 더미
COL_DUM = ['Profession','Var_1']
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)


# 데이터 나누기
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)


In [5]:
# 모델링
# 분류
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier()
modelRF.fit(X_tr, y_tr)

# 예측, 평가
y_pred_RF = modelRF.predict(X_val)

from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred_RF))

# 튜닝
from sklearn.model_selection import GridSearchCV
param = { 'n_estimators':[10,20,30,40,50],
            'max_depth':[3,5,7,9]}
  


              precision    recall  f1-score   support

           1       0.40      0.42      0.41       466
           2       0.34      0.32      0.33       474
           3       0.55      0.51      0.53       533
           4       0.62      0.68      0.65       527

    accuracy                           0.49      2000
   macro avg       0.48      0.48      0.48      2000
weighted avg       0.48      0.49      0.49      2000



In [6]:
# 제출
gs = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param, n_jobs=-1, cv=5)
result = gs.fit(X_tr, y_tr)
print('최적점수 : {}'.format(result.best_score_))
print('최적 파라미터 : {}'.format(result.best_params_))

final = RandomForestClassifier(n_estimators=50, max_depth=9)
final.fit(X_tr, y_tr)
y_pred_fi = final.predict(X_val)
print(classification_report(y_val, y_pred_fi))

pred=final.predict(X_test)
pd.DataFrame({'ID':X_test_ID, 'Segmentation':pred}).to_csv('submission.csv',index=False)

최적점수 : 0.532261521972133
최적 파라미터 : {'max_depth': 7, 'n_estimators': 40}
              precision    recall  f1-score   support

           1       0.44      0.50      0.46       466
           2       0.44      0.34      0.38       474
           3       0.62      0.62      0.62       533
           4       0.65      0.69      0.67       527

    accuracy                           0.55      2000
   macro avg       0.54      0.54      0.54      2000
weighted avg       0.54      0.55      0.54      2000

